In [1]:
import random, re
from collections import defaultdict
import itertools
import pycosat
import matplotlib.pyplot as plt
import pandas as pd
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer, BasicAer, IBMQ, execute
from qiskit.tools.visualization import plot_histogram
from qiskit.circuit.quantumcircuit import QuantumCircuit
from qiskit.circuit.library.standard_gates import ZGate
from qiskit import transpile
from qiskit.transpiler.passes import Unroller

In [2]:
def tcnfgen(n,k,horn=1):
    cnf = []
    def unique(l,k):
        t = random.randint(1,n)
        while(t in l):
            t = random.randint(1,n)
        return t
    r = (lambda : random.randint(0,1))
    def r_to_sign(x):
        if r() == 1:
            return x
        else:
            return -x
    for i in range(k):
        x = unique([],n)
        y = unique([x],n)
        z = unique([x, y],n)
        if horn:
            cnf.append([x, -y,-z])
        else:
            cnf.append([r_to_sign(x), r_to_sign(y),r_to_sign(z)])
    return cnf

In [3]:
def create_CNF(n, k):
    new_line = []
    line = tcnfgen(n, k)
    for clause in line:
        new_line.append(sorted(clause, key = abs))
    return new_line
def prepare_clause(circuit, clause):
    for element in clause:
        if element<0:
            circuit.x(abs(element))
    return circuit
def or_operator(circuit, clause, counter):
    order_clause = [abs(x) for x in clause]
    for element in order_clause:
        circuit.x(element)
    circuit.mcx(order_clause, counter)
    circuit.x(counter)
    for element in order_clause:
        circuit.x(element)
    return circuit
def oracle(n, k, CNF):
    circuit = QuantumCircuit(n+k+1)
    counter = n+1 
    for clause in CNF:
        prepare_clause(circuit, clause)
        or_operator(circuit, clause, counter)
        counter = counter + 1
        prepare_clause(circuit, clause)
    circuit.mcx(list(range(n+1, n+k+1)), 0)
    return circuit

In [4]:
CCCZ = ZGate().control(2)
def inversion_z(circuit,n):
    for i in range(1,n+1):
        circuit.h(i)

    for i in range(1,n+1):
        circuit.x(i)
    
    circuit.append(CCCZ, [1,2,3])

    for i in range(1,n+1):
        circuit.x(i)

    for i in range(1,n+1):
        circuit.h(i)
    return circuit

In [5]:
n = 3
k = 7
new_line = [[-1,-2,-3],[-1,-2,3],[-1,2,-3],[-1,2,3],[1,-2,-3],[1,-2,3],[1,2,-3]]
for sol in pycosat.itersolve(new_line):
    print(sol)

[-1, -2, -3]


In [6]:
grover_circuit = QuantumCircuit(n+k+1, n+k+1)

for i in range(1, n+1):
    grover_circuit.h(i)

iterations = 2
    
for i in range(iterations):
    grover_circuit += oracle(n, k, new_line)
    grover_circuit.z(0)
    grover_circuit += oracle(n, k, new_line).inverse()
    #grover_circuit.barrier()
    inversion_z(grover_circuit, n)
    #grover_circuit.barrier()

    
grover_circuit.measure([1,2,3],[1,2,3])

job = execute(grover_circuit, Aer.get_backend('qasm_simulator'), shots=10000)
#job = execute(mycircuit, backend= simulator_backend, shots=8192)
counts = job.result().get_counts(grover_circuit)
# print the reverse of the outcome
for outcome in counts:
    reverse_outcome = ''
    for i in outcome:
        reverse_outcome = i + reverse_outcome
    print(reverse_outcome, "is observed", counts[outcome], "times")
print("\n")


<ipython-input-6-fd5b23913f39>:9: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  grover_circuit += oracle(n, k, new_line)
<ipython-input-6-fd5b23913f39>:11: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  grover_circuit += oracle(n, k, new_line).inverse()


00010000000 is observed 71 times
00000000000 is observed 9448 times
00110000000 is observed 80 times
01110000000 is observed 81 times
00100000000 is observed 72 times
01100000000 is observed 84 times
01010000000 is observed 85 times
01000000000 is observed 79 times




In [7]:
grover_circuit.count_ops()

OrderedDict([('x', 304),
             ('mcx', 28),
             ('h', 15),
             ('mcx_gray', 4),
             ('measure', 3),
             ('z', 2),
             ('ccz', 2)])

In [8]:
qc = transpile(grover_circuit, basis_gates = ['sx', 'cx', 'x', 'rz', 'i'], optimization_level=2)

In [9]:
qc.draw()

global phase: 0
      ┌─────────┐┌────┐┌─────────┐                                             »
 q_0: ┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─────────────────────────────────────────────»
      ├─────────┤├────┤└┬───────┬┘                                             »
 q_1: ┤ Rz(π/2) ├┤ √X ├─┤ Rz(π) ├────■────────────────■─────────────────────■──»
      ├─────────┤├────┤┌┴───────┴─┐┌─┴─┐┌──────────┐┌─┴─┐                   │  »
 q_2: ┤ Rz(π/2) ├┤ √X ├┤ Rz(5π/8) ├┤ X ├┤ Rz(-π/8) ├┤ X ├──■────────────────┼──»
      ├─────────┤├────┤├──────────┤└───┘└──────────┘└───┘┌─┴─┐┌──────────┐┌─┴─┐»
 q_3: ┤ Rz(π/2) ├┤ √X ├┤ Rz(5π/8) ├──────────────────────┤ X ├┤ Rz(-π/8) ├┤ X ├»
      ├─────────┤├────┤├──────────┤                      └───┘└──────────┘└───┘»
 q_4: ┤ Rz(π/2) ├┤ √X ├┤ Rz(5π/8) ├────────────────────────────────────────────»
      ├─────────┤├────┤├──────────┤                                            »
 q_5: ┤ Rz(π/2) ├┤ √X ├┤ Rz(5π/8) ├────────────────────────────────────────────»
      ├─────────┤├────┤├──────────┤                                            »
 q_6: ┤ Rz(π/2) ├┤ √X ├┤ Rz(5π/8) ├────────────────────────────────────────────»
      ├─────────┤├────┤├──────────┤                                            »
 q_7: ┤ Rz(π/2) ├┤ √X ├┤ Rz(5π/8) ├────────────────────────────────────────────»
      ├─────────┤├────┤├──────────┤                                            »
 q_8: ┤ Rz(π/2) ├┤ √X ├┤ Rz(5π/8) ├────────────────────────────────────────────»
      ├─────────┤├────┤├──────────┤                                            »
 q_9: ┤ Rz(π/2) ├┤ √X ├┤ Rz(5π/8) ├────────────────────────────────────────────»
      ├─────────┤├────┤├──────────┤                                            »
q_10: ┤ Rz(π/2) ├┤ √X ├┤ Rz(5π/8) ├────────────────────────────────────────────»
      └─────────┘└────┘└──────────┘                                            »
c: 11/═════════════════════════════════════════════════════════════════════════»
                                                                               »
«                                                                             »
« q_0: ───────────────────────────────────────────────────────────────────────»
«                                                                             »
« q_1: ──────────────────────────────■────────────────────────────────────────»
«                                    │                                        »
« q_2: ─────────────■────────────────┼─────────────────────■──────────────────»
«      ┌─────────┐┌─┴─┐┌──────────┐┌─┴─┐                   │                  »
« q_3: ┤ Rz(π/8) ├┤ X ├┤ Rz(-π/8) ├┤ X ├──■────────────────┼───────────────■──»
«      └─────────┘└───┘└──────────┘└───┘┌─┴─┐┌──────────┐┌─┴─┐┌─────────┐┌─┴─┐»
« q_4: ─────────────────────────────────┤ X ├┤ Rz(-π/8) ├┤ X ├┤ Rz(π/8) ├┤ X ├»
«                                       └───┘└──────────┘└───┘└─────────┘└───┘»
« q_5: ───────────────────────────────────────────────────────────────────────»
«                                                                             »
« q_6: ───────────────────────────────────────────────────────────────────────»
«                                                                             »
« q_7: ───────────────────────────────────────────────────────────────────────»
«                                                                             »
« q_8: ───────────────────────────────────────────────────────────────────────»
«                                                                             »
« q_9: ───────────────────────────────────────────────────────────────────────»
«                                                                             »
«q_10: ───────────────────────────────────────────────────────────────────────»
«                                                                             »
«c: 11/═══════════════════════════════════════════════════════════════════════»
«                                      

In [10]:
qc.count_ops()

OrderedDict([('rz', 1999),
             ('cx', 1928),
             ('sx', 59),
             ('x', 42),
             ('measure', 3)])

In [11]:
qc.depth()

3260